In [18]:
'''
AI Assisted GL Mapping
Code 
'''

import csv
import pandas as pd
#import re


#Cleanup of data
def cleanup_data():
    #Converting the excel file into a pandas dataframe
    training_data = pd.read_excel('AI_POC - Updated.xlsx', sheet_name=0)
    test_data = pd.read_excel('AI_POC - Updated.xlsx', sheet_name=1)
    print(training_data.head())
    training_data['Gl_Account_Id']=training_data['Gl_Account_Id'].str.lower()
    training_data['Description']=training_data['Description'].str.lower()
    training_data['Account_Group']=training_data['Account_Group'].str.lower()
    training_data['GL_Nature']=training_data['GL_Nature'].str.lower()
    training_data['Grouping_Id_desc']=training_data['Grouping_Id_desc'].str.lower()
    training_data['Description'].replace('[^A-Za-z0-9 ]+', '', regex=True, inplace=True)
    training_data['Description'].replace('  ', ' ', regex=True, inplace=True)
    training_data['Account_Group'].replace('[^A-Za-z0-9 ]+', '', regex=True, inplace=True)
    training_data['Account_Group'].replace('  ', ' ', regex=True, inplace=True)
    training_data['GL_Nature'].replace('[^A-Za-z0-9 ]+', '', regex=True, inplace=True)
    training_data['Grouping_Id_desc'].replace('[^A-Za-z0-9 ]+', '', regex=True, inplace=True)
    print(training_data[:15])
    
    
cleanup_data()

                


   Taxonomy_Id  Coa_Master_Id  tenant_id  Template_Id Gl_Account_Id  \
0           39           6514     100004       130152        536001   
1           39           6514     100004       130152        536001   
2           39           6514     100004       130152        536001   
3           39           6514     100004       130152        536001   
4           39           6514     100004       130152        536002   

                Description Account_Group GL_Nature Grouping_Id_desc  \
0              Bank Charges  Bank Charges   Expense     Bank Charges   
1              Bank Charges  Bank Charges   Expense     Bank Charges   
2              Bank Charges  Bank Charges   Expense     Bank Charges   
3              Bank Charges  Bank Charges   Expense     Bank Charges   
4  Bank Guaratee Commission  Bank Charges   Expense     Bank Charges   

   Grouping_Id  Reverse_Flag GL_Nature.1  Level1_Grouping_Id  \
0      2100420             0     Expense             2100248   
1      21004